# Demo: Reactive vs Deliberative Agents

This demo compares reactive and deliberative agents navigating through different environments.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from collections import deque
import heapq

In [ ]:
class GridWorld:
    """A grid world with obstacles"""
    
    def __init__(self, width: int = 20, height: int = 20):
        self.width = width
        self.height = height
        self.grid = np.zeros((height, width))
        self.start = (1, 1)
        self.goal = (height - 2, width - 2)
    
    def add_obstacle(self, x: int, y: int):
        self.grid[y, x] = 1
    
    def add_wall(self, x1: int, y1: int, x2: int, y2: int):
        for x in range(min(x1, x2), max(x1, x2) + 1):
            for y in range(min(y1, y2), max(y1, y2) + 1):
                self.grid[y, x] = 1
    
    def is_valid(self, pos: tuple) -> bool:
        x, y = pos
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.grid[y, x] == 0
        return False
    
    def get_neighbors(self, pos: tuple) -> list:
        x, y = pos
        neighbors = []
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_pos = (x + dx, y + dy)
            if self.is_valid(new_pos):
                neighbors.append(new_pos)
        return neighbors

class ReactiveNavigator:
    """Reactive agent: Always moves toward goal, avoids immediate obstacles"""
    
    def __init__(self, world: GridWorld):
        self.world = world
        self.position = world.start
        self.path = [self.position]
    
    def move(self) -> bool:
        if self.position == self.world.goal:
            return True
        
        neighbors = self.world.get_neighbors(self.position)
        if not neighbors:
            return False
        
        # Greedy: move toward goal
        goal = self.world.goal
        best = min(neighbors, key=lambda p: abs(p[0] - goal[0]) + abs(p[1] - goal[1]))
        
        self.position = best
        self.path.append(self.position)
        return self.position == self.world.goal

class DeliberativeNavigator:
    """Deliberative agent: Plans path using A*"""
    
    def __init__(self, world: GridWorld):
        self.world = world
        self.position = world.start
        self.path = []
        self.plan = self._astar()
        self.step_idx = 0
    
    def _heuristic(self, a: tuple, b: tuple) -> float:
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def _astar(self) -> list:
        start = self.world.start
        goal = self.world.goal
        
        open_set = [(self._heuristic(start, goal), 0, start)]
        came_from = {}
        g_score = {start: 0}
        
        while open_set:
            _, _, current = heapq.heappop(open_set)
            
            if current == goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.append(start)
                return path[::-1]
            
            for neighbor in self.world.get_neighbors(current):
                tentative_g = g_score[current] + 1
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score = tentative_g + self._heuristic(neighbor, goal)
                    heapq.heappush(open_set, (f_score, tentative_g, neighbor))
        
        return []  # No path found
    
    def move(self) -> bool:
        if self.step_idx < len(self.plan):
            self.position = self.plan[self.step_idx]
            self.path.append(self.position)
            self.step_idx += 1
            return self.position == self.world.goal
        return True

In [ ]:
# Create world with obstacles
world = GridWorld(20, 20)

# Add walls that create a maze-like environment
world.add_wall(5, 0, 5, 12)
world.add_wall(10, 7, 10, 19)
world.add_wall(15, 0, 15, 10)

# Create agents
reactive = ReactiveNavigator(GridWorld(20, 20))
reactive.world = world  # Share the world
reactive.position = world.start
reactive.path = [world.start]

deliberative = DeliberativeNavigator(world)

# Run simulation
max_steps = 200
for _ in range(max_steps):
    reactive.move()
    if reactive.position == world.goal:
        break

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, agent, title in [(axes[0], reactive, 'Reactive Agent'),
                          (axes[1], deliberative, 'Deliberative Agent (A*)')]:
    ax.imshow(world.grid, cmap='binary', origin='lower')
    
    # Plot path
    path = np.array(agent.path if agent.path else agent.plan)
    if len(path) > 0:
        ax.plot(path[:, 0], path[:, 1], 'b-', linewidth=2, alpha=0.7)
    
    # Start and goal
    ax.scatter(*world.start, c='green', s=200, marker='o', label='Start', zorder=5)
    ax.scatter(*world.goal, c='red', s=200, marker='*', label='Goal', zorder=5)
    
    ax.set_title(f"{title}\nSteps: {len(agent.path) if agent.path else len(agent.plan)}")
    ax.legend()
    ax.set_xlim(-0.5, world.width - 0.5)
    ax.set_ylim(-0.5, world.height - 0.5)

plt.tight_layout()
plt.show()

print(f"Reactive agent steps: {len(reactive.path)}")
print(f"Deliberative agent steps: {len(deliberative.plan)}")

## Key Observations

1. **Reactive Agent**: May get stuck or take suboptimal paths because it only considers immediate neighbors
2. **Deliberative Agent**: Finds optimal path by planning ahead using A* algorithm
3. **Trade-off**: Deliberative is better but requires more computation upfront